In [4]:
import pandas as pd
from citeline.database.milvusdb import MilvusDB
from citeline.embedders import Embedder

db = MilvusDB()
print(db)

embedder = Embedder.create("Qwen/Qwen3-Embedding-0.6B", device="mps", normalize=True)

collection_name = "qwen06_chunks"

df = pd.read_json("../data/dataset/nontrivial_10.jsonl", lines=True)
df.head()

,source_doi,sent_original,sent_no_cit,sent_idx,citation_dois,pubdate,resolved_bibcodes,sent_cit_masked
0,10.1007/s00159-008-0010-0,"(Shang et al. 1998), do show unusual, faint fe...","(), do show unusual, faint features in their s...",159,[10.1086/311563],20080601,[1998ApJ...504L..23S],"([REF]), do show unusual, faint features in th..."
1,10.1016/j.newar.2004.03.017,Novak et al. (2003) studied the dust emission ...,studied the dust emission at 450 μm in the ga...,1329,[10.1086/368156],20040901,[2003ApJ...583L..83N],[REF] studied the dust emission at 450 μm in t...
2,10.1146/annurev-astro-091918-104430,The momentum per unit mass of stars formed del...,The momentum per unit mass of stars formed del...,542,[10.1086/317785],20190801,[2000ApJ...545..364M],The momentum per unit mass of stars formed del...
3,10.1146/annurev-astro-081309-130834,An example of the latter is the dormant blue s...,An example of the latter is the dormant blue s...,815,[10.48550/arXiv.astro-ph/9701042],20100901,[1997A&A...328..130P],An example of the latter is the dormant blue s...
4,10.1146/annurev.astro.37.1.127,Kaiser et al (1995) (see also Luppino Kaiser 1...,(see also for further developments) compute ...,282,"[10.1086/176071, 10.1086/303508, 10.1086/306102]",19990101,"[1995ApJ...449..460K, 1997ApJ...475...20L, 199...",[REF] (see also [REF] for further developments...


In [7]:
query_vectors = embedder(df["sent_no_cit"])
results = db.search(
    collection_name=collection_name,
    query_records=df.to_dict(orient="records"),
    query_vectors=query_vectors.tolist(),
    limit=5
)

In [11]:
for result in results[2]:
    print(result['metric'])

0.7363651990890503
0.7362618446350098
0.7282290458679199
0.727614164352417
0.7260739207267761


In [12]:
results_1 = results[0]
results_1_df = pd.DataFrame(results_1)
reciprocal_ranks = pd.Series([1 / (i + 1) for i in range(len(results_1_df))], index=results_1_df.index)
print(reciprocal_ranks)

0    1.000000
1    0.500000
2    0.333333
3    0.250000
4    0.200000
dtype: float64


In [ ]:
import json
# Open '../experiments/multiple_query_expansion/results/search_results.jsonl' and read in a batch at a time
rf = RankFuser({'reciprocal_rank': 1.0})
chunk_size = 1
def yield_chunk(filepath, chunk_size):
    """Yield lists of parsed JSON objects from a JSONL file in chunks."""
    chunk = []
    with open(filepath, "r") as f:
        for line in f:
            chunk.append(json.loads(line))
            if len(chunk) == chunk_size:
                yield chunk
                chunk = []
        if chunk:
            yield chunk

def find_perturbations():
    for chunk in yield_chunk("../experiments/multiple_query_expansion/results/search_results.jsonl", chunk_size):
        for row in chunk:
            query = pd.Series(row['record'])
            results = pd.DataFrame(row['results'])
            reranked_df = rf._rerank_single(query, results)
            is_monotonic_decreasing = reranked_df['metric'].is_monotonic_decreasing
            if not is_monotonic_decreasing:
                print("Scores are not monotonic decreasing for query")
                print(reranked_df['metric'].tolist())
                print(query)
                return query, results

query, results = find_perturbations()


Scores are not monotonic decreasing for query
[0.741113543510437, 0.714166522026062, 0.7163183093070984, 0.7073190212249756, 0.7024654150009155, 0.696847140789032, 0.6745415925979614, 0.695683479309082, 0.6684107184410095, 0.6861385107040405, 0.6521819829940796, 0.682979166507721, 0.6509355306625366, 0.6775891184806824, 0.6493569612503052, 0.6720894575119019, 0.6483639478683472, 0.6713389158248901, 0.6461080312728882, 0.6712968349456787, 0.6415587663650513, 0.6608102917671204, 0.638081431388855, 0.651623010635376, 0.636128842830658, 0.651499330997467, 0.6335941553115845, 0.6448408365249634, 0.6307758092880249, 0.6442087888717651, 0.6266939043998718, 0.6413660049438477, 0.6265087127685547, 0.6409002542495728, 0.6252028942108154, 0.6395514011383057, 0.6239104866981506, 0.6394235491752625, 0.6217306852340698, 0.6387937664985657, 0.6212885975837708, 0.6382416486740112, 0.6195082664489746, 0.6380873918533325, 0.6189847588539124, 0.6366219520568848, 0.6185566186904907, 0.6348631381988525, 0.

In [40]:
results['metric'].is_monotonic_decreasing

False